In [1]:
import random
import string
import numpy as np
import matplotlib.pyplot as plt

from skimage.io import imshow
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model, Sequential
from keras.layers import Input, Activation
from keras.layers.core import Dropout, Lambda, Dense, Flatten
from keras.layers.convolutional import Conv2D, Conv2DTranspose, Convolution2D
from keras.layers.pooling import MaxPooling2D,MaxPool2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras.optimizers import Adadelta

import tensorflow as tf
from sklearn.model_selection import train_test_split

/Users/andresregal/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
def captcha_acc(real, pred, seq_len=4, char_set=36):
    y_real = tf.reshape(real, [seq_len,char_set])
    y_pred = tf.reshape(pred, [seq_len,char_set])
    return 1 - tf.reduce_mean(tf.reduce_sum(tf.reduce_sum(tf.abs(y_real-y_pred,0),0)))

In [ ]:
def captcha_loss(real, pred, seq_len=4, char_set=36):
    y_real = tf.reshape(real, [seq_len,char_set])
    y_pred = tf.reshape(pred, [seq_len,char_set])
    return tf.reduce_mean(tf.reduce_sum(tf.reduce_sum(tf.abs(y_real-y_pred,0),0)))

In [49]:
def captcha_loss_K(real, pred,seq_len=4, char_set=36):
    y_real = K.argmax(K.reshape(real, [seq_len, char_set]))
    y_pred = K.argmax(K.reshape(pred, [seq_len, char_set]))
    return K.mean(tf.scalar_mul((1/8),K.sum(K.sum(K.abs(y_real-y_pred),0))))

In [50]:
def captcha_acc_K(real, pred, seq_len=4, char_set=36):
    y_real = K.argmax(K.reshape(real, [seq_len, char_set]))
    y_pred = K.argmax(K.reshape(pred, [seq_len, char_set]))
    return 1 - K.mean(tf.scalar_mul((1/8),K.sum(K.sum(K.abs(y_real-y_pred),0))))

In [15]:
x = np.random.rand(1,144)

In [37]:
K.argmax(K.reshape(x, [4,36]))

<tf.Tensor 'ArgMax:0' shape=(4,) dtype=int64>

In [36]:
K.one_hot(K.reshape(x, [4,36]),)

TypeError: one_hot() takes exactly 2 arguments (1 given)

In [16]:
pred = np.zeros((1,144))

In [17]:
x

array([[0.26009025, 0.768568  , 0.65861371, 0.44185471, 0.78997741,
        0.46789123, 0.23065634, 0.28705904, 0.05182795, 0.12433578,
        0.24537497, 0.16540133, 0.77794948, 0.91887509, 0.20883666,
        0.06519239, 0.98947829, 0.6034194 , 0.12110616, 0.19916396,
        0.62983391, 0.31711342, 0.09588346, 0.84930139, 0.2155018 ,
        0.66924382, 0.01515891, 0.83362416, 0.01132412, 0.17131031,
        0.5242167 , 0.43151275, 0.44954724, 0.34761759, 0.00270009,
        0.5087564 , 0.01621963, 0.70003648, 0.63001106, 0.70165678,
        0.60206207, 0.48508496, 0.61564605, 0.4039608 , 0.49969438,
        0.93349818, 0.68501404, 0.38266548, 0.29743401, 0.39434227,
        0.04612731, 0.30177309, 0.98453907, 0.73620842, 0.46165235,
        0.62796686, 0.52544531, 0.19464112, 0.25572528, 0.46198133,
        0.2941434 , 0.79819067, 0.17496085, 0.51414823, 0.44010276,
        0.56527639, 0.98350035, 0.65152407, 0.1945379 , 0.65680817,
        0.96611706, 0.1533441 , 0.00925671, 0.44

In [51]:
captcha_loss_K(x, pred)

<tf.Tensor 'Mean_3:0' shape=() dtype=int64>

In [4]:
X_train = np.load('data/train_inputs.npy')
y_train = np.load('data/train_targets.npy')

In [43]:
height = 60
width = 160
num_classes = 36
word_len = 4
learning_rate = 0.1

In [44]:
input_tensor = Input(shape=(height, width, 3))

x = input_tensor

x = Conv2D(32, 3, padding='same', activation='relu')(x)
x = Conv2D(32, 3, padding='same', activation='relu')(x)
x = Activation("relu")(x)
x = MaxPool2D((2, 2))(x)

x = Conv2D(64, 3, padding='same', activation='relu')(x)
x = Conv2D(64, 3, padding='same', activation='relu')(x)
x = Activation("relu")(x)
x = MaxPool2D((2, 2))(x)

x = Conv2D(128, 3, padding='same', activation='relu')(x)
x = Conv2D(128, 3, padding='same',activation='relu')(x)
x = Activation("relu")(x)
x = MaxPool2D((2, 2))(x)

x = Conv2D(256, 3, padding='same', activation='relu')(x)
x = Conv2D(256, 3, padding='same',activation='relu')(x)
x = Activation("relu")(x)
x = MaxPool2D((2, 2))(x)

x = Flatten()(x)
x = Dropout(0.25)(x)

x = [Dense(num_classes, activation='softmax', name='c%d'%(i+1))(x) for i in range(word_len)]
output = concatenate(x)

In [52]:
model = Model(inputs=input_tensor, outputs=output)

opt = Adadelta(lr=learning_rate)

model.compile(loss = captcha_loss_K, optimizer=opt, metrics=['accuracy'])

model.summary()

ValueError: Tensor conversion requested dtype int64 for Tensor with dtype float32: 'Tensor("concatenate_1_sample_weights_2:0", shape=(?,), dtype=float32)'

In [ ]:
earlystopper = EarlyStopping(patience=5, verbose=1)

checkpointer = ModelCheckpoint(filepath="weights.{epoch:02d}--{val_loss:.2f}-{val_acc:.4f}.hdf5", 
                               verbose=1, save_weights_only=True)

model.fit(X_train, y_train, epochs= 15, validation_split=0.1, callbacks=[earlystopper])

In [ ]:
X_test = np.load('data/test_inputs.npy')
y_test = np.load('data/test_targets.npy')

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
imshow(X_test[0])
imshow(X_test[6]);

In [ ]:
y_predicted = model.predict(X_test[0:7,:,:])

In [ ]:
max_ix = np.argmax(y_predicted[6].reshape((4,36)), axis=1)

In [ ]:
char_set = list(string.digits + string.ascii_letters)

In [ ]:
string_ = ''
for i in max_ix:
    string_ += char_set[i]

print(string_)

In [ ]:
def captcha_accuracy(y_test, y_pred)
    y_test = y_test.reshape((4,36))
    y_pred = y_pred.reshape((4,36))
    
    y_test - y_pred

In [ ]:
a = np.array([[0,0,0,1]
              [0,1,0,0]])

b = np.array([[0,0,0,1]
              [0,1,0,0]])

In [ ]:
a - b